In [1]:
import pandas as pd
import requests
import pickle

In [2]:
base_url = "https://disease.sh/v3/covid-19"

In [3]:
def get_global_update(url=base_url, save=False):

    """
    Args:
        url (str, optional): common/home part of url used to derive other urls from which we get required information. Defaults to base_url
        save (bool, optional): arg determing whether the content gathered/updated should be saved or not. Defaults to False
    Returns:
        (dict, dict): two dictionaries with information of cases updated most recently today and the last night respectively
    """

    global_daily_url = url + "/all"
    global_yesterday_url = url + "/all?yesterday"

    global_daily_update = requests.get(global_daily_url).json()
    global_yesterday_update = requests.get(global_yesterday_url).json()

    if save:
        with open("./global_updates.pkl", "wb") as file:
            pickle.dump(global_daily_update, file)
            pickle.dump(global_yesterday_update, file)

    return global_daily_update, global_yesterday_update


In [4]:
daily, yes = get_global_update()
print(daily, yes, sep="\n\n")

{'updated': 1630739495392, 'cases': 220655863, 'todayCases': 58601, 'deaths': 4567602, 'todayDeaths': 1306, 'recovered': 197163226, 'todayRecovered': 72552, 'active': 18925035, 'critical': 105922, 'casesPerOneMillion': 28308, 'deathsPerOneMillion': 586, 'tests': 3353454779, 'testsPerOneMillion': 427310.81, 'population': 7847811738, 'oneCasePerPeople': 0, 'oneDeathPerPeople': 0, 'oneTestPerPeople': 0, 'activePerOneMillion': 2411.5, 'recoveredPerOneMillion': 25123.34, 'criticalPerOneMillion': 13.5, 'affectedCountries': 223}

{'updated': 1630739495392, 'cases': 220655863, 'todayCases': 58601, 'deaths': 4567602, 'todayDeaths': 1306, 'recovered': 197163226, 'todayRecovered': 72552, 'active': 18925035, 'critical': 105922, 'casesPerOneMillion': 28308, 'deathsPerOneMillion': 586, 'tests': 3353454779, 'testsPerOneMillion': 427310.81, 'population': 7847811738, 'oneCasePerPeople': 0, 'oneDeathPerPeople': 0, 'oneTestPerPeople': 0, 'activePerOneMillion': 2411.5, 'recoveredPerOneMillion': 25123.34, 

In [5]:
def get_continent_update(url=base_url, save=False):
    """
    Args:
        url (string, optional): common/home part of url used to derive other urls from which we get required information. Defaults to base_url
        save (bool, optional): arg determing whether the content gathered/updated should be saved or not. Defaults to False
    Returns:
        pd.DataFrame: a dataframe with the most recent updated cases segregated continent wise
    """
    continents_url = url + "/continents"

    continents_updates = requests.get(continents_url).json()

    continents = [
        {
            "continent": i["continent"],
            "lat": i["continentInfo"]["lat"],
            "long": i["continentInfo"]["long"],
            "confirmed_cumulative": i["cases"],
            "confirmed_new": i["todayCases"],
            "deaths_cumulative": i["deaths"],
            "deaths_new": i["todayDeaths"],
            "recovered_cumulative": i["recovered"],
            "recovered_new": i["todayRecovered"],
            "active": i["active"],
            "critical": i["critical"],
            "updated": i["updated"],
        }
        for i in continents_updates
    ]

    continents_df = pd.DataFrame(continents, columns=continents[0].keys())

    if save:
        continents_df.to_csv("./continents.csv", index=False)

    return continents_df

In [6]:
cont = get_continent_update()
cont

,continent,lat,long,confirmed_cumulative,confirmed_new,deaths_cumulative,deaths_new,recovered_cumulative,recovered_new,active,critical,updated
0,North America,31.676827,-146.470747,48842117,18731,1006673,758,38074046,17261,9761398,33013,1630739496300
1,Asia,23.702727,62.375064,70908446,32045,1047512,488,66165997,45567,3694937,41375,1630739496357
2,South America,-15.655156,-100.748423,37032591,0,1134070,0,34851630,0,1046891,14971,1630739496358
3,Europe,25.771324,-35.601226,55756868,6048,1178280,55,50876559,9701,3702029,11856,1630739496359
4,Africa,1.738387,-16.309464,7945176,0,198801,0,7074996,0,671379,4474,1630739496360
5,Australia-Oceania,-8.659916,91.146985,169944,1777,2251,5,119292,23,48401,233,1630739496361


In [7]:
def get_country_update(url=base_url, save=False):
    """
    Args:
        url (string, optional): common/home part of url used to derive other urls from which we get required information. Defaults to base_url
        save (bool, optional): arg determing whether the content gathered/updated should be saved or not. Defaults to False
    Returns:
        pd.DataFrame: a dataframe with the most recent updated cases segregated country wise
    """

    countries_url = url + "/countries"

    countries_updates = requests.get(countries_url).json()

    countries = [
        {
            "country": i["country"],
            "lat": i["countryInfo"]["lat"],
            "long": i["countryInfo"]["long"],
            "confirmed_cumulative": i["cases"],
            "confirmed_new": i["todayCases"],
            "deaths_cumulative": i["deaths"],
            "deaths_new": i["todayDeaths"],
            "recovered_cumulative": i["recovered"],
            "recovered_new": i["todayRecovered"],
            "active": i["active"],
            "critical": i["critical"],
            "updated": i["updated"],
            "continent": i["continent"],
            "flag": i["countryInfo"]["flag"],
        }
        for i in countries_updates
    ]

    countries_df = pd.DataFrame(countries, columns=countries[0].keys())

    if save:
        countries_df.to_csv("./countries.csv", index=False)

    return countries_df


In [8]:
count =  get_country_update()
count.head()

,country,lat,long,confirmed_cumulative,confirmed_new,deaths_cumulative,deaths_new,recovered_cumulative,recovered_new,active,critical,updated,continent,flag
0,Afghanistan,33.0,65.0,153395,0,7128,0,111300,0,34967,1124,1630739495865,Asia,https://disease.sh/assets/img/flags/af.png
1,Albania,41.0,20.0,149117,0,2508,0,135321,0,11288,3,1630739495867,Europe,https://disease.sh/assets/img/flags/al.png
2,Algeria,28.0,3.0,197308,0,5373,0,134390,0,57545,42,1630739495779,Africa,https://disease.sh/assets/img/flags/dz.png
3,Andorra,42.5,1.6,15055,0,130,0,14874,0,51,4,1630739496159,Europe,https://disease.sh/assets/img/flags/ad.png
4,Angola,-12.5,18.5,48261,0,1248,0,43795,0,3218,8,1630739495977,Africa,https://disease.sh/assets/img/flags/ao.png


In [9]:
def get_time_series(url=base_url, save=False):
    """
    Args:
        url (string, optional): common/home part of url used to derive other urls from which we get required information. Defaults to base_url
        save (bool, optional): arg determing whether the content gathered/updated should be saved or not. Defaults to False
    Returns:
        (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame): 4 timeseries, 1 global (all 3 combined into 1) and 3 country wise (segregated into confirmed, recovered and deaths) respectively
    """
    ts_base_url = url + "/historical"
    global_ts_url = ts_base_url + "/all?lastdays=all"

    global_ts_update = requests.get(global_ts_url).json()

    global_ts_df = pd.DataFrame(
        global_ts_update, columns=global_ts_update.keys()
    ).transpose()

    country_wise_ts_url = ts_base_url + "?lastdays=all"

    country_wise_ts = requests.get(country_wise_ts_url).json()

    ts_index = [i["country"] for i in country_wise_ts]

    country_confirmed = [i["timeline"]["cases"] for i in country_wise_ts]
    country_recovered = [i["timeline"]["recovered"] for i in country_wise_ts]
    country_deaths = [i["timeline"]["deaths"] for i in country_wise_ts]

    country_confirmed_df = pd.DataFrame(
        country_confirmed, columns=country_confirmed[0].keys(), index=ts_index
    )
    country_confirmed_df = country_confirmed_df.groupby(
        country_confirmed_df.index, as_index=True
    ).sum()

    country_deaths_df = pd.DataFrame(
        country_deaths, columns=ts_index + country_deaths[0].keys(), index=ts_index
    )
    country_deaths_df = country_deaths_df.groupby(
        country_deaths_df.index, as_index=True
    ).sum()

    country_recovered_df = pd.DataFrame(
        country_recovered, columns=country_recovered[0].keys(), index=ts_index
    )
    country_recovered_df = country_recovered_df.groupby(
        country_recovered_df.index, as_index=True
    ).sum()

    if save:

        global_ts_df.to_csv("./global_ts.csv", index=True)
        country_confirmed_df.to_csv("./country_confirmed.csv", index=True)
        country_deaths_df.to_csv("./country_deaths.csv", index=True)
        country_recovered_df.to_csv("./country_recovered.csv", index=True)

    return global_ts_df, country_confirmed_df, country_deaths_df, country_recovered_df

In [10]:
global_ts_df, country_confirmed_df, country_deaths_df, country_recovered_df = get_time_series()
global_ts_df.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
cases,557,655,941,1433,2118,2927,5578,6167,8235,9927,...,213915140,214627582,215452320,215991185,216433910,217119865,217708315,218431264,219062869,219778828
deaths,17,18,26,42,56,82,131,133,171,213,...,4463912,4474638,4486312,4494714,4501435,4510359,4519444,4531889,4541463,4553371
recovered,30,32,39,42,56,65,108,127,145,225,...,0,0,0,0,0,0,0,0,0,0


In [11]:
country_confirmed_df.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,152722,152822,152960,152960,152960,153148,153220,153260,153306,153375
Albania,0,0,0,0,0,0,0,0,0,0,...,141365,142253,143174,144079,144847,145333,146387,147369,148222,149117
Algeria,0,0,0,0,0,0,0,0,0,0,...,193171,193674,194186,194671,195162,195574,196080,196527,196915,197308
Andorra,0,0,0,0,0,0,0,0,0,0,...,15014,15016,15025,15025,15025,15032,15033,15046,15052,15055
Angola,0,0,0,0,0,0,0,0,0,0,...,46539,46726,46929,47079,47168,47331,47544,47781,48004,48261


In [12]:
country_confirmed_df.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,152722,152822,152960,152960,152960,153148,153220,153260,153306,153375
Albania,0,0,0,0,0,0,0,0,0,0,...,141365,142253,143174,144079,144847,145333,146387,147369,148222,149117
Algeria,0,0,0,0,0,0,0,0,0,0,...,193171,193674,194186,194671,195162,195574,196080,196527,196915,197308
Andorra,0,0,0,0,0,0,0,0,0,0,...,15014,15016,15025,15025,15025,15032,15033,15046,15052,15055
Angola,0,0,0,0,0,0,0,0,0,0,...,46539,46726,46929,47079,47168,47331,47544,47781,48004,48261


In [13]:
country_recovered_df.tail()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
Vietnam,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yemen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zambia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zimbabwe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def get_vaccination_update(url=base_url, save=False):

    """
    Args:
        url (string, optional): common/home part of url used to derive other urls from which we get required information. Defaults to base_url
        save (bool, optional): arg determing whether the content gathered/updated should be saved or not. Defaults to False
    Returns:
        (pd.DataFrame, pd.DataFrame): 2 dataframes for vaccination updates, globally and segregated country wise respectively
    """
    vaccine_base_url = url + "/vaccine/coverage"
    vaccine_global_url = vaccine_base_url + "?lastdays=all"
    vac_global_update = requests.get(vaccine_global_url).json()

    vac_global_df = pd.DataFrame(
        vac_global_update, columns=vac_global_update.keys(), index=[0]
    )

    vaccine_country_url = vaccine_base_url + "/countries?lastdays=all&fullData=false"
    vac_country_update = requests.get(vaccine_country_url).json()

    vac_index = [i["country"] for i in vac_country_update]
    vac_country_values = [i["timeline"] for i in vac_country_update]

    vac_country_df = pd.DataFrame(
        vac_country_values, columns=vac_country_values[0].keys(), index=vac_index
    )

    if save:
        vac_global_df.to_csv("./vac_global.csv", index=False)
        vac_country_df.to_csv("./vac_country.csv", index=True)

    return vac_global_df, vac_country_df

In [15]:
vac_global_df, vac_country_df = get_vaccination_update()
vac_global_df.head()

,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
0,0,0,0,1,1,1,1,1,1,1,...,5095821663,5141063629,5191277650,5225537297,5266362495,5301766283,5339857230,5386420252,5416303706,5416303706


In [16]:
vac_country_df.tail()

,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,...,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21
Vietnam,0,0,0,0,0,0,0,0,0,0,...,18522203,18843004,19151122,19151122,19431093,19710560,19966724,20210381,20210381,20210381
Wallis and Futuna,0,0,0,0,0,0,0,0,0,0,...,9676,9676,9676,9676,9676,9676,9676,9676,9676,9676
Yemen,0,0,0,0,0,0,0,0,0,0,...,311483,311483,311483,311483,311483,311483,311483,311483,311483,311483
Zambia,0,0,0,0,0,0,0,0,0,0,...,558307,562630,567261,571356,574630,575032,578300,582006,588857,588857
Zimbabwe,0,0,0,0,0,0,0,0,0,0,...,4015517,4076205,4075995,4110354,4137853,4172657,4219824,4270430,4323735,4323735
